In [ ]:
import param
import parambokeh
from bokeh.io import output_notebook
import numpy as np
import pandas as pd

output_notebook()

The paramBokeh library provides an easy way to manipulate parameters on ``Parameterized`` using the widgets on bokeh server and within the notebook. In addition to controlling input parameters a common usecase for using widgets in the notebook is to dynamically control some visual display output. In addition to all the standard parameters supplied by the ``param`` library, ``paramBokeh`` also supplies so called ``View`` parameters, which render bokeh plot output in a widget area. The output parameters may be updated simply by setting the parameter on the class.

In the first simple example we will declare a Parameterized class with a ``Number`` parameter called magnitude and an ``HTML`` parameter which will let us display some arbitrary HTML. In this case we will simply generate a pandas dataframe with random data within the update method and use the ``to_html`` method to convert it to an HTML table. If we define the ``update`` method as the callback of the widgets the table will now update whenever the slider is dragged. To ensure that the output is drawn on initialization we set ``on_init=True``.

In [ ]:
class HTMLExample(param.Parameterized):
    
    magnitude = param.Number(1, bounds=(0, 10))

    output = parambokeh.view.HTML()
    
    def update(self, **kwargs):
        self.output = pd.DataFrame(np.random.rand(10,2)*self.magnitude).to_html()

example = HTMLExample(name='HTMLExample')
layout = parambokeh.Widgets(example, on_init=True, callback=example.update)

The ``HTML`` parameter accepts any arbitrary HTML string but for convenience paramBokeh also allows rendering bokeh and HoloViews plots using the ``Plot`` parameter. Note however that we can only replace a plot when deploying on bokeh server, within the notebook we may only update a plot:

Additionally we can declare the ``view_position``, which specifies where the viewing widget will be placed in relation to the input widgets:

In [ ]:
import holoviews as hv

class CurveExample(hv.streams.Stream):

    color = param.Color(default='#000000', precedence=0)

    element = param.ObjectSelector(default=hv.Curve,
                                   objects=[hv.Curve, hv.Scatter, hv.Area],
                                  precedence=0)

    amplitude = param.Number(default=2, bounds=(2, 5))
    
    frequency = param.Number(default=2, bounds=(1, 10))
    
    output = parambokeh.view.Plot()
    
    def view(self, *args, **kwargs):
        return self.element(self.amplitude*np.sin(np.linspace(0, np.pi*self.frequency)),
                        vdims=[hv.Dimension('y', range=(-5, 5))]).opts(style=dict(color=self.color))
    
    def event(self, **kwargs):
        if not self.output or any(k in kwargs for k in ['color', 'element']):
            self.output = hv.DynamicMap(self.view, streams=[self], cache_size=0)
        else:
            super(CurveExample, self).event(**kwargs)

example = CurveExample(name='HoloViews Example')
parambokeh.Widgets(example, callback=example.event, on_init=True, view_position='right')